In [1]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

CPU times: user 8.22 s, sys: 5.94 s, total: 14.2 s
Wall time: 8.07 s


In [2]:
# Khai báo đường dẫn đến kết quả phân loại và kết quả thực tế của địa phương
KD_path = "ThuanHoa/KhoanhDat_new/ThuanHoa_TKDD2022.shp"
KetQuaPhanLoaiDat = "KetQuaPhanLoaiDat_15-09.tif"

In [14]:
# khai báo các loại đất ứng với các mã đất phân loại được
CODE_MAP = {
    "BHK": 2,
    "CLN": 3,
    "DGD": 6,
    "DGT": 6,
    "DNL": 6,
    "DRA": 6,
    "DSH": 6,
    "DTL": 5,
    "DTS": 6,
    "DYT": 6,
    "LUC": 1,
    "NKH": 3,
    "NTD": 6,
    "NTS": 4,
    "ONT": 6,
    "SKC": 6,
    "SKX": 6,
    "SON": 5,
    "TMD": 6,
    "TON": 6,
    "TSC": 6,
    "RPH": 7,
}

label_mapping = {
    "Lua tom": 0,
    "Lua": 1,
    "Cay hang nam": 2,
    "Cay lau nam": 3,
    "Thuy san": 4,
    "Song": 5,
    "Dat xay dung": 6,
    "Rung": 7
}

tenchu_to_label_mapping = {
    "BHK": "Cay hang nam",   
    "CLN": "Cay lau nam",    
    "DGD": "Dat xay dung",   
    "DGT": "Dat xay dung",   
    "DNL": "Dat xay dung",   
    "DRA": "Dat xay dung",   
    "DSH": "Dat xay dung",   
    "DTL": "Song",           
    "DTS": "Dat xay dung",   
    "DYT": "Dat xay dung",   
    "LUC": "Lua",            
    "NKH": "Cay lau nam",    
    "NTD": "Dat xay dung",   
    "NTS": "Thuy san",       
    "ONT": "Dat xay dung",   
    "SKC": "Dat xay dung",   
    "SKX": "Dat xay dung",   
    "SON": "Song",           
    "TMD": "Dat xay dung",   
    "TON": "Dat xay dung",   
    "TSC": "Dat xay dung",
    "RPH": "Rung"
}

# Khai báo các nhãn phân loại đất ứng với 3 loại đất chính
HT_MAP = {
    "NN": {"name": "Đất Nông Nghiệp", "data": [1, 2, 3, 4]},
    "PNN": {"name": "Đất Phi Nông Nghiệp", "data": [6]},
    "TQ": {"name": "Đất Thổ Quả", "data": [15]},
}

In [52]:
def test(KD_path, KetQuaPhanLoaiDat, CODE_MAP, label_mapping):
    gdf = gpd.read_file(KD_path, crs="EPSG:9209")
    polygons = gdf.geometry.values
    labels_before = gdf.tenchu.values
    output_image = rioxarray.open_rasterio(KetQuaPhanLoaiDat)
    
    array_list = []
    result = []  
    mapped_array_list = []  

    for i in range(len(polygons)):
        po = polygons[i]
        lb_before = labels_before[i]
        
        # Kiểm tra và gán "TQ" nếu có nhiều mã
        if ";" in lb_before:
            lb_before = "TQ"
        
        # Lấy mã từ CODE_MAP
        code_lb = CODE_MAP.get(lb_before, 15)
        
        try:
            qr = output_image.rio.clip([po], "EPSG:9209")
            changed_points = qr.where(qr != float(code_lb), np.nan)  

            # Kiểm tra có thay đổi không
            if changed_points.notnull().any():
                label_after_values = changed_points.values.flatten()

                for value in label_after_values:
                    if not np.isnan(value):
                        # Tìm label_after_key dựa trên value
                        label_after_key = next((key for key, val in label_mapping.items() if val == int(value)), None)
                        lb_mapped = tenchu_to_label_mapping.get(lb_before, None)
                        if lb_mapped is None:
                            print(f"Lỗi: Không tìm thấy ánh xạ cho nhãn '{lb_before}' trong tenchu_to_label_mapping.")
                            continue

                        result.append({
                            'label_before': lb_mapped,
                            'label_after': label_after_key
                        })
                        
                        mapped_array_list.append({
                            'image': changed_points,
                            'label_before': lb_mapped,
                            'label_after': label_after_key
                        })
                
                array_list.append(changed_points)  # Chỉ thêm vào array_list nếu có thay đổi
        except Exception as e:
            print(f"Lỗi xử lý polygon {i}: {e}")

    return array_list, result, mapped_array_list


In [72]:
output_json = "sankey_data.json"
sankey_data = CalcSankeyData(
    KD_path, 
    KetQuaPhanLoaiDat, 
    CODE_MAP, 
    label_mapping, 
    tenchu_to_label_mapping, 
    output_json
)

# Xem nội dung JSON xuất ra
sankey_data

Lỗi xử lý polygon 113: No data found in bounds.
Lỗi xử lý polygon 548: No data found in bounds.
Lỗi xử lý polygon 729: No data found in bounds.
Dữ liệu Sankey đã được lưu tại: sankey_data.json


[{'from': 'Cay hang nam', 'to': 'Cay lau nam', 'flow': 2237},
 {'from': 'Cay hang nam', 'to': 'Dat xay dung', 'flow': 155},
 {'from': 'Cay hang nam', 'to': 'Lua', 'flow': 1487},
 {'from': 'Cay hang nam', 'to': 'Lua tom', 'flow': 12},
 {'from': 'Cay hang nam', 'to': 'Rung', 'flow': 275},
 {'from': 'Cay hang nam', 'to': 'Thuy san', 'flow': 15},
 {'from': 'Cay lau nam', 'to': 'Cay hang nam', 'flow': 2255},
 {'from': 'Cay lau nam', 'to': 'Dat xay dung', 'flow': 3997},
 {'from': 'Cay lau nam', 'to': 'Lua', 'flow': 2657},
 {'from': 'Cay lau nam', 'to': 'Lua tom', 'flow': 85},
 {'from': 'Cay lau nam', 'to': 'Rung', 'flow': 779},
 {'from': 'Cay lau nam', 'to': 'Thuy san', 'flow': 17},
 {'from': 'Dat xay dung', 'to': 'Cay hang nam', 'flow': 1677},
 {'from': 'Dat xay dung', 'to': 'Cay lau nam', 'flow': 3516},
 {'from': 'Dat xay dung', 'to': 'Lua', 'flow': 2987},
 {'from': 'Dat xay dung', 'to': 'Lua tom', 'flow': 11},
 {'from': 'Dat xay dung', 'to': 'Rung', 'flow': 987},
 {'from': 'Dat xay dung',

In [ ]:
def tif_to_shp(tif_path, shp_path):
    with rasterio.open(tif_path) as src:
        # Read the first band with resampling
        image = src.read(1, resampling=Resampling.bilinear).astype('float32')  # Convert to float32
        mask = image != src.nodata  # Create a mask for non-no-data pixels

        # Vectorize the raster image into polygons (converting pixels to polygons)
        results = (
            {'geometry': shape(s), 'properties': {'raster_val': float(v)}}
            for s, v in shapes(image, mask=mask, transform=src.transform)
        )

        # Define schema for the shapefile
        schema = {
            'geometry': 'Polygon',
            'properties': {'raster_val': 'float'},
        }

        # Define coordinate reference system (CRS)
        crs = src.crs.to_wkt() if src.crs else 'EPSG:4326'

        # Open the shapefile for writing
        with fiona.open(shp_path, 'w', driver='ESRI Shapefile', schema=schema, crs=crs) as shp:
            for result in results:
                shp.write(result)

    print(f"Saved shapefile to {shp_path}")

# Example usage:
tif_path = "KetQuaPhanLoaiDat_15-09.tif"
shp_path = "SaveShapeFile/Test.shp"
tif_to_shp(tif_path, shp_path)


In [37]:
def combine_tif_and_test(KD_path, KetQuaPhanLoaiDat, CODE_MAP, label_mapping, tenchu_to_label_mapping, output_shapefile, xa_mapping_file):
    # Tải thông tin xaid từ file CSV (hoặc Excel nếu cần)
    xa_data = pd.read_csv(xa_mapping_file)

    # Tạo shapefile từ TIFF
    temp_shp_path = "temp_tif_shapefile.shp"
    tif_to_shp(KetQuaPhanLoaiDat, temp_shp_path)

    # Đọc shapefile từ TIFF
    gdf_tif = gpd.read_file(temp_shp_path)
    
    # Kiểm tra CRS của shapefile TIFF
    if gdf_tif.crs != "EPSG:9209":
        print(f"CRS hiện tại của TIFF: {gdf_tif.crs}. Chuyển sang EPSG:9209.")
        gdf_tif = gdf_tif.to_crs("EPSG:9209")

    # Đọc shapefile ban đầu
    gdf_kd = gpd.read_file(KD_path)
    
    # Kiểm tra CRS của shapefile ban đầu
    if gdf_kd.crs != "EPSG:9209":
        print(f"CRS hiện tại của shapefile KD: {gdf_kd.crs}. Chuyển sang EPSG:9209.")
        gdf_kd = gdf_kd.to_crs("EPSG:9209")

    # Tạo danh sách để lưu kết quả
    result_data = []

    for _, row_kd in gdf_kd.iterrows():
        kd_geom = row_kd['geometry']
        label_before = row_kd['tenchu']
        xaid = row_kd.get('xaid', None)  # Lấy giá trị xaid nếu có
        thuaid = row_kd.get('thuaid', None)  # Lấy giá trị thuaid nếu có

        # Kiểm tra và gán "TQ" nếu nhãn có nhiều mã
        if ";" in label_before:
            label_before = "TQ"

        # Lấy mã từ CODE_MAP
        code_lb = CODE_MAP.get(label_before, 15)
        lb_mapped = tenchu_to_label_mapping.get(label_before, "Unknown")
        
        if lb_mapped == "Unknown":
            print(f"Lỗi: Không tìm thấy ánh xạ cho nhãn '{label_before}' trong tenchu_to_label_mapping.")
            continue

        # Tra cứu thông tin xã từ xaid
        if xaid is not None:
            xa_info = xa_data[xa_data['Mã'] == xaid]
            if not xa_info.empty:
                ten_xa = xa_info['Tên'].values[0]
                ten_quan = xa_info['Quận Huyện'].values[0]
                ten_tinh = xa_info['Tỉnh / Thành Phố'].values[0]
            else:
                ten_xa = ten_quan = ten_tinh = "Unknown"
        else:
            ten_xa = ten_quan = ten_tinh = "Unknown"

        # Lấy các polygon từ TIFF có giao cắt với polygon hiện tại
        for _, row_tif in gdf_tif.iterrows():
            tif_geom = row_tif['geometry']
            raster_val = row_tif['raster_val']

            # Kiểm tra giao cắt
            if kd_geom.intersects(tif_geom):
                intersection = kd_geom.intersection(tif_geom)

                # Kiểm tra raster_val không phải NaN trước khi xử lý
                if not np.isnan(raster_val):
                    # Tìm nhãn sau dựa trên giá trị raster
                    label_after = next((key for key, val in label_mapping.items() if val == int(raster_val)), "Unknown")

                    # Chỉ lưu các thay đổi (nhãn trước và sau khác nhau)
                    if lb_mapped != label_after:
                        # Suy ra nhãn từ label_after bằng cách tra cứu CODE_MAP
                        category_after = next((key for key, val in CODE_MAP.items() if val == label_mapping.get(label_after, -1)), "Unknown")

                        result_data.append({
                            "geometry": intersection,
                            "label_before": lb_mapped,
                            "label_after": label_after,
                            "category_after": category_after,
                            "xaid": xaid,
                            "thuaid": thuaid,
                            "ten_xa": ten_xa,
                            "ten_quan": ten_quan,
                            "ten_tinh": ten_tinh
                        })

    # Chuyển danh sách kết quả thành GeoDataFrame
    result_gdf = gpd.GeoDataFrame(result_data, crs="EPSG:9209")

    # Chuyển hệ tọa độ của kết quả cuối cùng sang EPSG:4326
    result_gdf_4326 = result_gdf.to_crs("EPSG:4326")

    # Lưu shapefile đã chuyển đổi
    result_gdf_4326.to_file(output_shapefile, driver="ESRI Shapefile")
    print(f"Shapefile kết quả đã được lưu tại (hệ tọa độ EPSG:4326): {output_shapefile}")

    # Xóa file tạm
    try:
        for ext in [".shp", ".shx", ".dbf", ".prj", ".cpg"]:
            temp_file = temp_shp_path.replace(".shp", ext)
            if os.path.exists(temp_file):
                os.remove(temp_file)
                print(f"Đã xóa file tạm: {temp_file}")
    except Exception as e:
        print(f"Lỗi khi xóa file tạm: {e}")

    # return result_gdf_4326


In [15]:
output_shapefile = "SaveShapeFile/Test.shp"
xa_mapping_file = "xa_mapping.csv"
combine_tif_and_test(KD_path, KetQuaPhanLoaiDat, CODE_MAP, label_mapping, tenchu_to_label_mapping, output_shapefile, xa_mapping_file)

Saved shapefile to temp_tif_shapefile.shp
CRS hiện tại của TIFF: EPSG:32648. Chuyển sang EPSG:9209.
CRS hiện tại của shapefile KD: PROJCS["Transverse_Mercator",GEOGCS["WGS 84",DATUM["D_unknown",SPHEROID["WGS84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105.5],PARAMETER["scale_factor",0.9999],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]. Chuyển sang EPSG:9209.
Đã xóa file tạm: temp_tif_shapefile.shp
Đã xóa file tạm: temp_tif_shapefile.shx
Đã xóa file tạm: temp_tif_shapefile.dbf
Đã xóa file tạm: temp_tif_shapefile.prj
Đã xóa file tạm: temp_tif_shapefile.cpg


In [ ]:
output_shapefile = "KetQuaPhanLoaiSHP/output_raster_to_shp_with_mapping.shp"
temp = gpd.read_file(output_shapefile, crs="EPSG:9209")
temp.head(20).geometry[19]

In [16]:
temp = gpd.read_file("SaveShapeFile/Test.shp", crs="EPSG:9209")
temp.head(20)

,KH_before,LB_before,KH_after,LB_after,xaid,thuaid,ten_xa,ten_quan,ten_tinh,geometry
0,SKC,Dat xay dung,BHK,Cay hang nam,31576,7620220493,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86822 9.68080, 105.86823 9.68082..."
1,SKC,Dat xay dung,BHK,Cay hang nam,31576,7620220493,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86888 9.68078, 105.86888 9.68069..."
2,SKC,Dat xay dung,BHK,Cay hang nam,31576,7620220493,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86806 9.68066, 105.86806 9.68054..."
3,SKC,Dat xay dung,BHK,Cay hang nam,31576,7620220493,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86815 9.68052, 105.86815 9.68060..."
4,SKC,Dat xay dung,BHK,Cay hang nam,31576,7620220493,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86906 9.68060, 105.86906 9.68051..."
5,SKC,Dat xay dung,LUC,Lua,31576,7620220493,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86833 9.68047, 105.86833 9.68051..."
6,SKC,Dat xay dung,CLN,Cay lau nam,31576,7620220493,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86860 9.68080, 105.86860 9.68078..."
7,SKC,Dat xay dung,LUC,Lua,31576,7620220493,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86906 9.68031, 105.86906 9.68032..."
8,ONT,Dat xay dung,CLN,Cay lau nam,31576,7620220456,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86894 9.68141, 105.86897 9.68141..."
9,CLN,Cay lau nam,BHK,Cay hang nam,31576,7620220465,Xã Thuận Hòa,Huyện Châu Thành,Tỉnh Sóc Trăng,"POLYGON ((105.86915 9.68114, 105.86915 9.68105..."


In [77]:
import geopandas as gpd
import numpy as np
import pandas as pd
import os
from shapely.geometry import mapping

def CompareToSHP_EachHTCode(KD_path, KetQuaPhanLoaiDat, CODE_MAP, label_mapping, tenchu_to_label_mapping, HT_MAP, xa_mapping_file, output_folder):
    # Tải thông tin xaid từ file CSV
    xa_data = pd.read_csv(xa_mapping_file)

    # Tạo shapefile từ TIFF
    temp_shp_path = "temp_tif_shapefile.shp"
    tif_to_shp(KetQuaPhanLoaiDat, temp_shp_path)

    # Đọc shapefile từ TIFF
    gdf_tif = gpd.read_file(temp_shp_path)
    
    # Đảm bảo CRS thống nhất
    if gdf_tif.crs != "EPSG:9209":
        gdf_tif = gdf_tif.to_crs("EPSG:9209")
    gdf_kd = gpd.read_file(KD_path)
    if gdf_kd.crs != "EPSG:9209":
        gdf_kd = gdf_kd.to_crs("EPSG:9209")

    # Xử lý từng hiện trạng trong HT_MAP
    for ht_name, ht_data in HT_MAP.items():
        print(f"Processing for current state: {ht_name}")
        ht_result_data = []

        for _, row_kd in gdf_kd.iterrows():
            kd_geom = row_kd['geometry']
            label_before = row_kd['tenchu']
            xaid = row_kd.get('xaid', None)
            thuaid = row_kd.get('thuaid', None)

            # Gán "TQ" nếu nhãn có nhiều mã
            if ";" in label_before:
                label_before = "TQ"

            # Lấy mã từ CODE_MAP
            code_lb = CODE_MAP.get(label_before, 15)
            lb_mapped = tenchu_to_label_mapping.get(label_before, "Unknown")
            
            if lb_mapped == "Unknown":
                continue

            # Tra cứu thông tin xã từ xaid
            if xaid is not None:
                xa_info = xa_data[xa_data['Mã'] == xaid]
                if not xa_info.empty:
                    ten_xa = xa_info['Tên'].values[0]
                    ten_quan = xa_info['Quận Huyện'].values[0]
                    ten_tinh = xa_info['Tỉnh / Thành Phố'].values[0]
                else:
                    ten_xa = ten_quan = ten_tinh = "Unknown"
            else:
                ten_xa = ten_quan = ten_tinh = "Unknown"

            # Xử lý các đối tượng thuộc hiện trạng hiện tại
            if code_lb in ht_data["data"]:
                for _, row_tif in gdf_tif.iterrows():
                    tif_geom = row_tif['geometry']
                    raster_val = row_tif['raster_val']

                    # Kiểm tra giao cắt
                    if kd_geom.intersects(tif_geom):
                        intersection = kd_geom.intersection(tif_geom)

                        if not np.isnan(raster_val):
                            label_after = next((key for key, val in label_mapping.items() if val == int(raster_val)), "Unknown")
                            if lb_mapped != label_after:
                                category_after = next((key for key, val in CODE_MAP.items() if val == label_mapping.get(label_after, -1)), "Unknown")
                                ht_result_data.append({
                                    "geometry": intersection,
                                    "label_before": lb_mapped,
                                    "label_after": label_after,
                                    "category_after": category_after,
                                    "xaid": xaid,
                                    "thuaid": thuaid,
                                    "ten_xa": ten_xa,
                                    "ten_quan": ten_quan,
                                    "ten_tinh": ten_tinh
                                })

        # Lưu kết quả hiện trạng ra shapefile riêng
        if ht_result_data:
            ht_result_gdf = gpd.GeoDataFrame(ht_result_data, crs="EPSG:9209")
            ht_output_path = os.path.join(output_folder, f"{ht_name}_changes.shp")
            ht_result_gdf.to_file(ht_output_path, driver="ESRI Shapefile")
            print(f"Shapefile for {ht_name} saved at: {ht_output_path}")
        else:
            print(f"No changes found for {ht_name}. Skipping.")

    # Xóa file tạm
    try:
        for ext in [".shp", ".shx", ".dbf", ".prj", ".cpg"]:
            temp_file = temp_shp_path.replace(".shp", ext)
            if os.path.exists(temp_file):
                os.remove(temp_file)
                print(f"Temporary file deleted: {temp_file}")
    except Exception as e:
        print(f"Error deleting temporary file: {e}")


In [78]:
import os

output_folder = "output_shapefiles"
os.makedirs(output_folder, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

combine_tif_and_test_by_ht(KD_path, KetQuaPhanLoaiDat, CODE_MAP, label_mapping, tenchu_to_label_mapping, HT_MAP, xa_mapping_file, output_folder)


Saved shapefile to temp_tif_shapefile.shp
Processing for current state: NN


/tmp/ipykernel_965/1499816764.py:89: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ht_result_gdf.to_file(ht_output_path, driver="ESRI Shapefile")


Shapefile for NN saved at: output_shapefiles/NN_changes.shp
Processing for current state: PNN


/tmp/ipykernel_965/1499816764.py:89: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ht_result_gdf.to_file(ht_output_path, driver="ESRI Shapefile")


Shapefile for PNN saved at: output_shapefiles/PNN_changes.shp
Processing for current state: TQ
No changes found for TQ. Skipping.
Temporary file deleted: temp_tif_shapefile.shp
Temporary file deleted: temp_tif_shapefile.shx
Temporary file deleted: temp_tif_shapefile.dbf
Temporary file deleted: temp_tif_shapefile.prj
Temporary file deleted: temp_tif_shapefile.cpg


In [21]:
def raster_to_shp_with_mapping(raster_path, boundary_path, label_mapping, tenchu_to_label_mapping, output_shapefile):
    # Đọc shapefile vùng boundary
    boundary_gdf = gpd.read_file(boundary_path)
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs  # CRS của raster
        transform = src.transform

    # Chuyển shapefile boundary về cùng CRS với raster
    if boundary_gdf.crs != raster_crs:
        print(f"Chuyển boundary CRS từ {boundary_gdf.crs} sang {raster_crs}.")
        boundary_gdf = boundary_gdf.to_crs(raster_crs)

    # Lấy vùng hình học từ boundary
    boundary_geom = [geom for geom in boundary_gdf.geometry]

    # Mở tệp raster và cắt theo boundary
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)  # Đọc dải raster đầu tiên (giả sử dữ liệu trên 1 band)
        
        # Kiểm tra và chuyển đổi kiểu dữ liệu nếu cần
        if raster_data.dtype not in ['int16', 'int32', 'uint8', 'uint16', 'float32']:
            print(f"Chuyển đổi kiểu dữ liệu từ {raster_data.dtype} sang int32.")
            raster_data = raster_data.astype('int32')  # Chuyển về kiểu int32
        
        try:
            # Cắt raster theo boundary
            raster_clipped, clipped_transform = mask(src, boundary_geom, crop=True)
            raster_data_clipped = raster_clipped[0]  # Lấy band đầu tiên

            # Kiểm tra kiểu dữ liệu raster sau khi cắt
            if raster_data_clipped.dtype not in ['int16', 'int32', 'uint8', 'uint16', 'float32']:
                print(f"Chuyển đổi kiểu dữ liệu cắt từ {raster_data_clipped.dtype} sang int32.")
                raster_data_clipped = raster_data_clipped.astype('int32')

        except ValueError as e:
            print("Lỗi: Không có vùng giao giữa boundary và raster.")
            print(e)
            return

    # Chuyển raster đã cắt thành vector (shapefile)
    shapes_generator = shapes(raster_data_clipped, transform=clipped_transform)
    features = []
    for geom, value in shapes_generator:
        if not np.isnan(value):  # Bỏ qua các giá trị null
            label = label_mapping.get(int(value), "Unknown")  # Mapping giá trị raster thành nhãn
            if label != "Unknown":  
                # Ánh xạ từ label sang tenchu_code
                tenchu_code = next((code for code, lbl in tenchu_to_label_mapping.items() if lbl == label), None)
                features.append({"geometry": shape(geom), "label": label, "raster_value": int(value), "tenchu": tenchu_code})

    # Tạo GeoDataFrame từ các đối tượng vector
    gdf = gpd.GeoDataFrame(features, crs=raster_crs)

    # Chuyển GeoDataFrame về hệ tọa độ EPSG:4326
    gdf = gdf.to_crs(epsg=4326)
    print(f"Chuyển đổi GeoDataFrame sang hệ tọa độ EPSG:4326.")

    # Lưu GeoDataFrame dưới dạng shapefile
    gdf.to_file(output_shapefile, driver="ESRI Shapefile")
    print(f"Shapefile đã được lưu tại: {output_shapefile}")



# Ví dụ sử dụng
label_mapping = {
    1: "Lua tom",
    2: "Lua",
    3: "Cay hang nam",
    4: "Cay lau nam",
    5: "Thuy san",
    6: "Song",
    7: "Dat xay dung",
    8: "Rung"
}

boundary_path = "ThuanHoa/region/ST_ThuanHoa_Boundaryofficially.shp"
output_shapefile = "KetQuaPhanLoaiSHP/output_raster_to_shp_with_mapping.shp"

raster_to_shp_with_mapping(KetQuaPhanLoaiDat, boundary_path, label_mapping, tenchu_to_label_mapping, output_shapefile)


Chuyển boundary CRS từ EPSG:9209 sang EPSG:32648.
Chuyển đổi kiểu dữ liệu từ float64 sang int32.
Chuyển đổi kiểu dữ liệu cắt từ float64 sang int32.


/tmp/ipykernel_111/1448566265.py:23: RuntimeWarning: invalid value encountered in cast
  raster_data = raster_data.astype('int32')  # Chuyển về kiểu int32


Chuyển đổi GeoDataFrame sang hệ tọa độ EPSG:4326.


/tmp/ipykernel_111/1448566265.py:59: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shapefile, driver="ESRI Shapefile")


Shapefile đã được lưu tại: KetQuaPhanLoaiSHP/output_raster_to_shp_with_mapping.shp


In [22]:
def CalcSankeyData(KD_path, KetQuaPhanLoaiDat, CODE_MAP, label_mapping, tenchu_to_label_mapping, output_json):
    gdf = gpd.read_file(KD_path, crs="EPSG:9209")
    polygons = gdf.geometry.values
    labels_before = gdf.tenchu.values
    output_image = rioxarray.open_rasterio(KetQuaPhanLoaiDat)

    result = []  

    for i in range(len(polygons)):
        po = polygons[i]
        lb_before = labels_before[i]
        
        # Kiểm tra và gán "TQ" nếu có nhiều mã
        if ";" in lb_before:
            lb_before = "TQ"
        
        # Lấy mã từ CODE_MAP
        code_lb = CODE_MAP.get(lb_before, 15)
        
        try:
            qr = output_image.rio.clip([po], "EPSG:9209")
            changed_points = qr.where(qr != float(code_lb), np.nan)  

            # Kiểm tra có thay đổi không
            if changed_points.notnull().any():
                label_after_values = changed_points.values.flatten()

                for value in label_after_values:
                    if not np.isnan(value):
                        # Tìm label_after_key dựa trên value
                        label_after_key = next((key for key, val in label_mapping.items() if val == int(value)), None)
                        lb_mapped = tenchu_to_label_mapping.get(lb_before, None)
                        if lb_mapped is None:
                            print(f"Lỗi: Không tìm thấy ánh xạ cho nhãn '{lb_before}' trong tenchu_to_label_mapping.")
                            continue

                        result.append({
                            'from': lb_mapped,
                            'to': label_after_key
                        })
                        
        except Exception as e:
            print(f"Lỗi xử lý polygon {i}: {e}")

    # Chuyển kết quả thành DataFrame
    df = pd.DataFrame(result)

    sankey_data = df.groupby(['from', 'to']).size().reset_index(name='flow')
    sankey_data = sankey_data[sankey_data['flow'] > 100]

    # Tính phần trăm cho mỗi thay đổi
    # sankey_data['Percentage'] = sankey_data['flow'] / sankey_data['flow'].sum()

    # Chuyển dữ liệu thành dạng JSON
    # sankey_dict = sankey_data.to_dict(orient="records")
    # with open(output_json, "w", encoding="utf-8") as json_file:
    #     json.dump(sankey_dict, json_file, ensure_ascii=False, indent=4)
    
    print(f"Dữ liệu Sankey đã được lưu tại: {output_json}")
    return sankey_data


In [23]:
output_json = ""
CalcSankeyData(KD_path, KetQuaPhanLoaiDat, CODE_MAP, label_mapping, tenchu_to_label_mapping, output_json)

Lỗi xử lý polygon 113: No data found in bounds.
Lỗi xử lý polygon 548: No data found in bounds.
Lỗi xử lý polygon 729: No data found in bounds.
Dữ liệu Sankey đã được lưu tại: 


,from,to,flow
0,Cay hang nam,Cay lau nam,2237
1,Cay hang nam,Dat xay dung,155
2,Cay hang nam,Lua,1487
4,Cay hang nam,Rung,275
6,Cay lau nam,Cay hang nam,2255
7,Cay lau nam,Dat xay dung,3997
8,Cay lau nam,Lua,2657
10,Cay lau nam,Rung,779
12,Dat xay dung,Cay hang nam,1677
13,Dat xay dung,Cay lau nam,3516
